In [154]:
import lxml
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from requests import request
import json
import urllib
from lxml import etree, html as lhtml
import re

In [77]:
base_url = "https://shop.relod.ru/catalog-products/4577/?sort=PROPERTY_RATING&order=desc&PAGEN_1="
pages_url = []

In [78]:
for number_of_page in tqdm(range(1, 251)):
    curr_url = base_url + str(number_of_page)
    html_text = request('GET', curr_url)
    tree = lhtml.fromstring(html_text.text)
    pages_url.append(tree.xpath('//div[@class ="bxr-element-name"]/a/@href'))

100%|██████████| 250/250 [00:52<00:00,  4.80it/s]


In [84]:
flat_list = [item for sublist in pages_url for item in sublist]

In [88]:
assert(len(flat_list) == len(set(flat_list)))

All urls are unique, so we don't have any dublicates.

In [125]:
flat_list[2]

'/catalog-products/zorro/'

In [206]:
base_book_url = "https://shop.relod.ru"
test_url = base_book_url + flat_list[2]
html_test = request('GET', test_url)
tree = lhtml.fromstring(html_test.text)
checker = tree.xpath('//tr[@itemprop="additionalProperty"]/td/span') # получили свойства

In [161]:
regex = re.compile('"PRICE_FORMAT": "([0-9]*)')

In [162]:
checker_new = tree.xpath('//script')
for obj_new in checker_new:
    if obj_new.text is not None:
        if "var ptvSettings" in obj_new.text:
            print(int(regex.findall(obj_new.text)[0]))

146


In [207]:
checker_images = tree.xpath('//div[@class ="slick-track"]')

In [208]:
checker_images

[]